<a href="https://colab.research.google.com/github/ShuangAnatoli/Machine-Learning/blob/main/DecisionTrees/FraudDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Download Dataset

NotADirectoryError: ignored

In [ ]:
!pip install snapml
!pip install opendatasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import opendatasets as od
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize, StandardScaler
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.metrics import roc_auc_score
import time
import warnings
from __future__ import print_function
warnings.filterwarnings('ignore')

In [ ]:
od.download("https://www.kaggle.com/mlg-ulb/creditcardfraud")


NotADirectoryError: ignored

#Exploratory Data Analysis

In [ ]:
df = pd.read_csv("creditcard.csv")

In [ ]:
print("There are " + str(len(df)) + " observations in the credit card fraud dataset.")
print("There are " + str(len(df.columns)) + " variables in the dataset.")
df

There are 55551 observations in the credit card fraud dataset.
There are 31 variables in the dataset.


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0.0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0.0
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0.0
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0.0
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55546,47004,1.283418,0.354874,0.158519,0.607335,-0.186818,-0.865444,0.084609,-0.186195,0.042276,...,-0.310652,-0.887568,0.084508,-0.012270,0.267755,0.116476,-0.022729,0.027233,0.89,0.0
55547,47005,-0.969584,-0.514812,1.010076,-0.492493,-1.074442,0.812377,1.739011,0.069684,-0.741379,...,0.137958,-0.378729,0.989131,-0.311335,-0.302619,0.720578,-0.171671,0.034326,424.80,0.0
55548,47005,-0.448083,0.936043,1.308078,0.424255,0.141240,-0.224215,1.114183,-0.286478,-0.144553,...,0.084264,0.345878,-0.222427,0.052293,0.093368,-0.387899,-0.354380,-0.223041,55.75,0.0
55549,47005,0.616391,-0.770376,-0.306186,1.374006,-0.177289,0.186992,0.417697,0.073573,-0.009001,...,0.182098,-0.039148,-0.409170,-0.315762,0.626198,-0.254219,-0.042365,0.044234,293.00,0.0
